In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import scipy.optimize

matplotlib.use("tkAgg")


def energy(z, r, delta_rho, g):
    return 4*np.pi * r**3 * delta_rho * g * z / 3


def poisson_mean(mean0, z, r, delta_rho, g, k, T):
    # print(mean0, energy(z, r, delta_rho, g))
    return mean0 * np.exp(-energy(z, r, delta_rho, g) / (k*T))


def ln_L(k, mean0, z, n, r, delta_rho, g, T):
    mean_i = poisson_mean(mean0, z, r, delta_rho, g, k, T)
    # return np.sum(n * np.log(mean_i) - mean_i)
    # This should result in less precision loss
    return np.sum(n * (np.log(mean0) - energy(z, r, delta_rho, g) / (k*T)) - mean_i)


def main():
    # Natural constants
    g = 9.81  # m/s^2
    k_true = 1.38064852e-23  # m^2 kg s^-2 K^-1
    # g = 9.81e6  # µm/s^2
    # k_true = 1.38064852e-23 * (1e6)**2 * 1000  # µm^2 g s^-2 K^-1
    print("k should be in these units:", k_true)

    # System parameters
    r = 0.52e-6  # m
    rho_mastic = 1063  # kg/m^3
    rho_water = 997.0474  # kg/m^3 (approx., from Wikipedia)
    # r = 0.52  # µm
    # rho_mastic = 1.063e-12  # g/µm^3
    # rho_water = 0.9970474e-12  # g/µm^3
    delta_rho = rho_mastic - rho_water
    T = 293  # K
    focus_z = 1e-6  # m
    # focus_z = 1  # µm

    # Data
    z = np.array([0, 6, 12, 18])*1e-6
    n = np.array([1880, 940, 530, 305])
    # N = n.size
    mean0 = n[0]

    fig: plt.Figure = plt.figure(figsize=(11.7, 8.3))
    fig.tight_layout()
    ax1: plt.Axes = fig.add_subplot(131)
    ax2: plt.Axes = fig.add_subplot(132)
    ax3: plt.Axes = fig.add_subplot(133)

    ax1.bar(z, n, focus_z)
    ax1.set_xlabel("z (m)")
    ax1.set_ylabel("n")

    # Part i)
    print("\nPart i)")
    k_arr = np.linspace(1e-24, 1e-22, 10000)
    # k_arr = np.linspace(1e-9, 1e-8, 10000)
    ln_L_arr = np.empty_like(k_arr)
    for i in range(k_arr.size):
        ln_L_arr[i] = ln_L(k_arr[i], mean0, z, n, r, delta_rho, g, T)
    k_est = k_arr[ln_L_arr.argmax()]
    print("Estimated k:", k_est)

    # With the default method this results in the following error both with using m and µm as units
    # "Desired error not necessarily achieved due to precision loss."
    sol = scipy.optimize.minimize(
        (lambda *args: -ln_L(*args)),
        x0=np.array([k_est], dtype=np.longdouble),
        args=(mean0, z.astype(np.longdouble), n.astype(np.longdouble), r, delta_rho, g, T),
        # Using the Nelder-Mead algorithm helps with the precision problems
        # https://stackoverflow.com/questions/24767191/scipy-is-not-optimizing-and-returns-desired-error-not-necessarily-achieved-due
        method="Nelder-Mead"
    )
    k_sol = sol.x[0]
    ln_L_k_sol = ln_L(k_sol, mean0, z, n, r, delta_rho, g, T)
    print(sol)
    sol_min = scipy.optimize.root_scalar(
        (lambda *args: ln_L_k_sol - ln_L(*args) - 1/2),
        args=(mean0, z.astype(np.longdouble), n.astype(np.longdouble), r, delta_rho, g, T),
        bracket=(0.5 * k_sol, k_sol)
    )
    sol_max = scipy.optimize.root_scalar(
        (lambda *args: ln_L_k_sol - ln_L(*args) - 1/2),
        args=(mean0, z.astype(np.longdouble), n.astype(np.longdouble), r, delta_rho, g, T),
        bracket=(k_sol, 1.5*k_sol)
    )
    print(sol_min)
    print(sol_max)
    for ax in (ax2, ax3):
        ax.plot(k_arr, ln_L_arr)
        ax.scatter(k_est, ln_L(k_est, mean0, z, n, r, delta_rho, g, T), c="r")
        ax.axvline(sol_min.root)
        ax.axvline(sol_max.root)
        ax.set_xlabel("k")
        ax.set_ylabel("-ln L")
    ax3.set_xlim(k_sol - 1.5*(k_sol - sol_min.root), k_sol + 1.5*(sol_max.root - k_sol))

    # TODO estimate uncertainty graphically

    # Part ii)
    print("\nPart ii)")
    sol2 = scipy.optimize.minimize(
        (lambda x, *args: -ln_L(x[0], x[1], *args)),
        x0=np.array([k_est, mean0], dtype=np.longdouble),
        args=(z.astype(np.longdouble), n.astype(np.longdouble), r, delta_rho, g, T),
        method="Nelder-Mead"
    )
    print(sol2)
    # TODO estimate uncertainty graphically

    # Part iii)
    print("\nPart iii)")
    R = 8.314  # J/(mol*K)
    N_A = R/k_true  # TODO replace with sol2 results
    print("N_A:", N_A)

    plt.show()


if __name__ == "__main__":
    main()


k should be in these units: 1.38064852e-23

Part i)
Estimated k: 1.2366336633663368e-23
 final_simplex: (array([[1.23711672e-23],
       [1.23663366e-23]]), array([-22020.13878746, -22020.13873002]))
           fun: -22020.138787455016
       message: 'Optimization terminated successfully.'
          nfev: 16
           nit: 8
        status: 0
       success: True
             x: array([1.23711672e-23])
      converged: True
           flag: 'converged'
 function_calls: 2
     iterations: 1
           root: 1.2371167233910892e-23
      converged: True
           flag: 'converged'
 function_calls: 2
     iterations: 1
           root: 1.2371167233910892e-23

Part ii)
 final_simplex: (array([[1.25616213e-23, 1.84494446e+03],
       [1.25616221e-23, 1.84494453e+03],
       [1.25616228e-23, 1.84494446e+03]]), array([-22020.52420427, -22020.52420427, -22020.52420427]))
           fun: -22020.524204271147
       message: 'Optimization terminated successfully.'
          nfev: 86
           

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Attempting to set identical left == right == 1.2371167233910892e-23 results in singular transformations; automatically expanding.
